In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [4]:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'
referer = 'https://www.resilience.org/latest-articles/'
headers = {'User-Agent': user_agent, 'referer':referer}

In [6]:
topic_page = 'https://grist.org/all-topics/'
response = requests.get(topic_page, headers=headers)

In [8]:
soup = BeautifulSoup(response.text, 'html.parser')

In [20]:
topics = ['https://grist.org' + e['href'] for e in soup.find_all('a', {'class':'topic-listing__topic-link'})]

In [80]:
article_links = []
for topic in topics:
    
    response = requests.get(topic, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    last_page = int(soup.find_all('li', {'class':'pagination__page'})[-1].text.replace("\n", ""))

    for page in range(1, last_page+1):
        url = topic + 'page/{}/'.format(page)

        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        new_links = [e['href'] for e in soup.find_all('a', {'class':'tease__link'})]
        article_links.extend(new_links)

IndexError: list index out of range

In [ ]:
article_links_unique = set(article_links)

In [ ]:
print(len(article_links))
print(len(article_links_unique))

In [56]:
grist = {}

for article in article_links:
    grist[article] = get_article_data(article)

In [55]:
def get_article_data(article):

    art_dict = {}
    
    response = requests.get(article, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    art_dict['title']    = soup.find('h1', {'class':'topper-headings__title'}).text
    art_dict['subtitle'] = soup.find('h2', {'class':'topper-headings__subtitle'}).text
    art_dict['author']   = soup.find('span', {'class':'contributor-info__name'}).text
    art_dict['date']     = soup.find('dd', {'class':'article-meta__item-value'}).text
    art_dict['text']     = "".join([p.text for p in soup.find('div', {'class':'article-body js-hang-punc'}).find_all('p')])
    
    return art_dict

In [57]:
for key in grist.keys():
    print(grist[key]['title'])


      This coastal town needs federal aid for climate adaptation. Instead it’ll get a military truck.
    

      Lessons from a year of reporting on climate solutions in Cascadia
    

      The Biden administration said its drilling-lease spree in the Gulf was court-ordered. It wasn’t.
    

      A Missouri gas company figured out how to keep its illegal pipeline running
    

      Who gets help after a hurricane or flood? FEMA will start tracking it by race
    

      Toxic Tides: Climate change expected to cause 400 toxic California sites to flood by 2100
    

      Interior’s new oil and gas leasing plan sidesteps climate action
    

      Black and Latino neighborhoods pay more for energy despite far lower emissions
    
